# Prepare Required Dependencies

In [1]:
# Install these if needed
# !pip install diffusers transformers accelerate

In [47]:
import os
from diffusers import StableDiffusionInpaintPipeline, StableDiffusionImg2ImgPipeline
import torch
import torchvision.transforms as transforms
import cv2 as cv
from PIL import Image

In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [12]:
PATCH_SIZE = 8

def generate_mask_from_image(img):
    m = torch.zeros([img.shape[0], img.shape[1]])
    for i in range(0, m.shape[0], PATCH_SIZE):
        for j in range(0, m.shape[1], PATCH_SIZE):
            m[i:i + PATCH_SIZE, j:j + PATCH_SIZE][3:5, 3:5] = 1.0

    return m

In [13]:
input_transform = transforms.Compose([
    transforms.PILToTensor()
])

output_transform = transforms.Compose([
    transforms.ToPILImage()
])

In [38]:
img_path = "missing_pixels/balloon.bmp"

In [39]:
img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
img = cv.resize(img, (256, 256), cv.INTER_LINEAR)

In [40]:
if not os.path.exists("./images"):
    os.makedirs("./images")

In [41]:
cv.imwrite("./images/input.png", img)

True

In [42]:
mask = generate_mask_from_image(img)
mask_image = output_transform(mask)

In [43]:
mask_image.save("./images/mask.png")

# Model Prediction

In [48]:
input_img = Image.open("./images/input.png")
mask_img = Image.open("./images/mask.png")

In [4]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu")
print("device: ", device)

device:  mps


## Inpainting model

In [5]:
inpainting_model = "runwayml/stable-diffusion-inpainting"
inpainting_pipe = StableDiffusionInpaintPipeline.from_pretrained(inpainting_model).to(device)

vae/diffusion_pytorch_model.safetensors not found
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [53]:
prompt = "high , black and white"
out_img = inpainting_pipe(prompt=prompt, image=input_img, mask_image=mask_img).images[0]
out_img.resize((256, 256)).save("./images/inpainting_output.png")

  0%|          | 0/50 [00:00<?, ?it/s]